In [1]:
import pathlib

import numpy as np
import pandas as pd
import sys
sys.path.append('../SocGFM/src')

from preprocessing_util import coRetweet


In [2]:
# Script Hyper-Parameters
DATASET_NAME = 'cuba'
extract_largest_connected_component = True

# Script constants
CONTROL_FILE_IDX, IO_FILE_IDX = 0, 1
filename_dict = {'UAE_sample': ['control_driver_tweets_uae_082019.jsonl', 'uae_082019_tweets_csv_unhashed.csv'],
                 'cuba': ['control_driver_tweets_cuba_082020.jsonl', 'cuba_082020_tweets_csv_unhashed.csv']}

# Basic definitions
base_dir = pathlib.Path.cwd().parent / 'SocGFM'
data_dir = base_dir / 'data' / 'raw'
processed_data_dir = base_dir / 'data' / 'processed'

# Read raw data
io_df = pd.read_csv(data_dir / DATASET_NAME / filename_dict[DATASET_NAME][IO_FILE_IDX], sep=",")
control_df = pd.read_json(data_dir / DATASET_NAME / filename_dict[DATASET_NAME][CONTROL_FILE_IDX], lines=True)

# Build Co-Retweet
control, treated = control_df.copy(), io_df.copy()
control_column_names = ['id', 'user', 'retweeted_status']
treated_column_names = ['userid', 'tweetid', 'retweet_tweetid']
control = control[control_column_names]
treated = treated[treated_column_names]
coRetweet_network = coRetweet(control, treated)


/tmp/ipykernel_2646833/1408084759.py:16: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  io_df = pd.read_csv(data_dir / DATASET_NAME / filename_dict[DATASET_NAME][IO_FILE_IDX], sep=",")


Restrict the analysis to users present in the coRT network

In [111]:
valid_user_ids = list(coRetweet_network.nodes())
control_df['userid'] = control_df['user'].apply(lambda x: int(dict(x)['id']))  # Retrieve user ID
filtered_control_df = control_df[control_df['userid'].astype(str).isin(valid_user_ids)]
filtered_treated_df = io_df[io_df['userid'].astype(str).isin(valid_user_ids)]


In [112]:
filtered_control_df.head()

,id,full_text,source,lang,user,geo,coordinates,place,in_reply_to_status_id,in_reply_to_screen_name,in_reply_to_user_id,created_at,entities,retweeted_status,extended_entities,context_annotations,userid
0,24148603702,RT @amoleiro: coincido: la aclaratoria de fide...,Twitter Web Client,es,"{'location': 'USA', 'profile_image_url': 'http...",NaN,NaN,None,NaN,None,NaN,2010-09-10 23:59:56+00:00,"{'annotations': [{'start': 42, 'end': 46, 'pro...","{'id': '24147670831', 'full_text': 'coincido: ...",NaN,NaN,41929478
1,24148595306,@yghome ¿Fidel estaba en directo en TV? me lo ...,Twitter Web Client,es,"{'location': 'USA', 'profile_image_url': 'http...",NaN,NaN,None,NaN,ggohom,71024346.000,2010-09-10 23:59:48+00:00,"{'hashtags': [{'text': 'Cuba', 'indices': [76,...",NaN,NaN,NaN,41929478
2,24148518366,"RT @mrperozo: RT @OrlandoOchoa: Así es ""@amole...",Twitter Web Client,es,"{'location': 'USA', 'profile_image_url': 'http...",NaN,NaN,None,NaN,None,NaN,2010-09-10 23:58:38+00:00,"{'annotations': [{'start': 79, 'end': 83, 'pro...","{'id': '24148373806', 'full_text': 'RT @Orland...",NaN,NaN,41929478
3,24143912305,RT @Lady_Liberty: Patriots! #FollowFriday #tco...,Twitter Web Client,lv,"{'location': 'USA', 'profile_image_url': 'http...",NaN,NaN,None,NaN,None,NaN,2010-09-10 22:48:44+00:00,"{'hashtags': [{'text': 'FollowFriday', 'indice...","{'id': '24141870726', 'full_text': 'Patriots! ...",NaN,NaN,41929478
4,24143826652,RT @blogsdecuba: Castro recula: ‘Es el capital...,Twitter Web Client,es,"{'location': 'USA', 'profile_image_url': 'http...",NaN,NaN,None,NaN,None,NaN,2010-09-10 22:47:23+00:00,"{'annotations': [{'start': 17, 'end': 22, 'pro...","{'id': '24143748382', 'full_text': 'Castro rec...",NaN,NaN,41929478


In [113]:
filtered_treated_df.head()

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,user_profile_url,follower_count,following_count,account_creation_date,...,retweet_tweetid,latitude,longitude,quote_count,reply_count,like_count,retweet_count,hashtags,urls,user_mentions
0,489569193772466176,543803256,Fernando Pérez Hernández🇨🇺,FernandoPerezHd,Cienfuegos,No soy Político ni pretendo convertirme en un ...,NaN,3971,3540,2012-04-02,...,489568125386383360.000,absent,absent,0,0,0,0,"['Cuba', 'CubaVa', 'CubaSi', 'YoaniFraude']",['http://ln.is/bit.ly/0WlGy'],[587848359]
1,420612879717789696,543803256,Fernando Pérez Hernández🇨🇺,FernandoPerezHd,Cienfuegos,No soy Político ni pretendo convertirme en un ...,NaN,3971,3540,2012-04-02,...,420611331008364544.000,absent,absent,0,0,0,0,"['Cuba', 'CubaVa', 'CubaSi']",['http://tinyurl.com/n3yqs5u'],[587848359]
2,368409820048916481,543803256,Fernando Pérez Hernández🇨🇺,FernandoPerezHd,Cienfuegos,No soy Político ni pretendo convertirme en un ...,NaN,3971,3540,2012-04-02,...,368408605067718656.000,absent,absent,0,0,0,0,"['Cuba', 'CubaVa', 'CubaSi']",['http://bit.ly/1bDB6Lj'],[587848359]
3,419160883110359040,543803256,Fernando Pérez Hernández🇨🇺,FernandoPerezHd,Cienfuegos,No soy Político ni pretendo convertirme en un ...,NaN,3971,3540,2012-04-02,...,419158255345344512.000,absent,absent,0,0,0,0,"['Cuba', 'CubaVa', 'CubaSi']",['http://ln.is/bit.ly/Z3ATI'],[587848359]
4,642077163173347329,543803256,Fernando Pérez Hernández🇨🇺,FernandoPerezHd,Cienfuegos,No soy Político ni pretendo convertirme en un ...,NaN,3971,3540,2012-04-02,...,642069667062771712.000,absent,absent,0,0,0,0,"['cuba', 'YoSoyCuba', 'PensandoAmericas']",['http://bit.ly/1Og4ZX3'],[602879083]


Get some basic statistics

In [44]:
control_users_tweets = filtered_control_df.userid.unique()
treated_users_tweets = filtered_treated_df.userid.unique()

assert len(np.union1d(control_users_tweets, treated_users_tweets)) == len(valid_user_ids), len(valid_user_ids)-len(np.union1d(control_users_tweets, treated_users_tweets))


In [61]:
control_users_tweets_stats = filtered_control_df.groupby('userid').count()['id']
treated_users_tweets_stats = filtered_treated_df.groupby('userid').count()['tweetid']


In [74]:
control_users_tweets_stats.describe()

count    4377.000000
mean       94.235549
std       155.071145
min        20.000000
25%        48.000000
50%        88.000000
75%        99.000000
max      6399.000000
Name: id, dtype: float64

In [79]:
treated_users_tweets_stats.describe()

count       429.000000
mean      11185.505828
std       26084.025503
min          28.000000
25%         826.000000
50%        3853.000000
75%       10631.000000
max      269516.000000
Name: tweetid, dtype: float64

In [86]:
filtered_control_df['tweet_length'] = filtered_control_df['full_text'].apply(lambda x: len(x))
filtered_treated_df['tweet_length'] = filtered_treated_df['tweet_text'].apply(lambda x: len(x))


/tmp/ipykernel_776725/489325356.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_control_df['tweet_length'] = filtered_control_df['full_text'].apply(lambda x: len(x))
/tmp/ipykernel_776725/489325356.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_treated_df['tweet_length'] = filtered_treated_df['tweet_text'].apply(lambda x: len(x))


In [91]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
filtered_treated_df['tweet_length'].describe()


count   4798582.000
mean        134.328
std          43.131
min           1.000
25%         136.000
50%         140.000
75%         140.000
max        1020.000
Name: tweet_length, dtype: float64

In [92]:
filtered_control_df['tweet_length'].describe()


count   412469.000
mean       122.225
std         38.091
min          1.000
25%        108.000
50%        139.000
75%        140.000
max        986.000
Name: tweet_length, dtype: float64

#### Data Cleaning:
- Removing mentions
- Removing URLs, i.e., "http:" links
- Removing Emojis
- Removing hashtags keeping the text, i.e. the "#" sign


In [114]:
import re
import emoji

def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = emoji.replace_emoji(tweet, '') #Remove Emojis
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    return tweet


In [115]:
filtered_control_df['full_text_cleaned'] = filtered_control_df['full_text'].apply(lambda x: cleaner(x))
filtered_treated_df['tweet_text_cleaned'] = filtered_treated_df['tweet_text'].apply(lambda x: cleaner(x))


/tmp/ipykernel_776725/3447196453.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_control_df['full_text_cleaned'] = filtered_control_df['full_text'].apply(lambda x: cleaner(x))
/tmp/ipykernel_776725/3447196453.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_treated_df['tweet_text_cleaned'] = filtered_treated_df['tweet_text'].apply(lambda x: cleaner(x))


In [116]:
filtered_control_df.head()


,id,full_text,source,lang,user,geo,coordinates,place,in_reply_to_status_id,in_reply_to_screen_name,in_reply_to_user_id,created_at,entities,retweeted_status,extended_entities,context_annotations,userid,full_text_cleaned
0,24148603702,RT @amoleiro: coincido: la aclaratoria de fide...,Twitter Web Client,es,"{'location': 'USA', 'profile_image_url': 'http...",NaN,NaN,None,NaN,None,NaN,2010-09-10 23:59:56+00:00,"{'annotations': [{'start': 42, 'end': 46, 'pro...","{'id': '24147670831', 'full_text': 'coincido: ...",NaN,NaN,41929478,RT : coincido: la aclaratoria de fidel sobre e...
1,24148595306,@yghome ¿Fidel estaba en directo en TV? me lo ...,Twitter Web Client,es,"{'location': 'USA', 'profile_image_url': 'http...",NaN,NaN,None,NaN,ggohom,71024346.000,2010-09-10 23:59:48+00:00,"{'hashtags': [{'text': 'Cuba', 'indices': [76,...",NaN,NaN,NaN,41929478,"¿Fidel estaba en directo en TV? me lo perdí, s..."
2,24148518366,"RT @mrperozo: RT @OrlandoOchoa: Así es ""@amole...",Twitter Web Client,es,"{'location': 'USA', 'profile_image_url': 'http...",NaN,NaN,None,NaN,None,NaN,2010-09-10 23:58:38+00:00,"{'annotations': [{'start': 79, 'end': 83, 'pro...","{'id': '24148373806', 'full_text': 'RT @Orland...",NaN,NaN,41929478,"RT : RT : Así es "": coincido: la aclaratoria d..."
3,24143912305,RT @Lady_Liberty: Patriots! #FollowFriday #tco...,Twitter Web Client,lv,"{'location': 'USA', 'profile_image_url': 'http...",NaN,NaN,None,NaN,None,NaN,2010-09-10 22:48:44+00:00,"{'hashtags': [{'text': 'FollowFriday', 'indice...","{'id': '24141870726', 'full_text': 'Patriots! ...",NaN,NaN,41929478,RT Liberty: Patriots! FollowFriday tcot D
4,24143826652,RT @blogsdecuba: Castro recula: ‘Es el capital...,Twitter Web Client,es,"{'location': 'USA', 'profile_image_url': 'http...",NaN,NaN,None,NaN,None,NaN,2010-09-10 22:47:23+00:00,"{'annotations': [{'start': 17, 'end': 22, 'pro...","{'id': '24143748382', 'full_text': 'Castro rec...",NaN,NaN,41929478,RT : Castro recula: ‘Es el capitalismo el que ...


In [117]:
filtered_treated_df.head()


,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,user_profile_url,follower_count,following_count,account_creation_date,...,latitude,longitude,quote_count,reply_count,like_count,retweet_count,hashtags,urls,user_mentions,tweet_text_cleaned
0,489569193772466176,543803256,Fernando Pérez Hernández🇨🇺,FernandoPerezHd,Cienfuegos,No soy Político ni pretendo convertirme en un ...,NaN,3971,3540,2012-04-02,...,absent,absent,0,0,0,0,"['Cuba', 'CubaVa', 'CubaSi', 'YoaniFraude']",['http://ln.is/bit.ly/0WlGy'],[587848359],RT : Cuba CubaVa Alemania expulsa al jefe de I...
1,420612879717789696,543803256,Fernando Pérez Hernández🇨🇺,FernandoPerezHd,Cienfuegos,No soy Político ni pretendo convertirme en un ...,NaN,3971,3540,2012-04-02,...,absent,absent,0,0,0,0,"['Cuba', 'CubaVa', 'CubaSi']",['http://tinyurl.com/n3yqs5u'],[587848359],"RT : Cuba CubaVa 2014… más revolución, más fem..."
2,368409820048916481,543803256,Fernando Pérez Hernández🇨🇺,FernandoPerezHd,Cienfuegos,No soy Político ni pretendo convertirme en un ...,NaN,3971,3540,2012-04-02,...,absent,absent,0,0,0,0,"['Cuba', 'CubaVa', 'CubaSi']",['http://bit.ly/1bDB6Lj'],[587848359],RT : Cuba CubaVa Presidente checo anuncia posi...
3,419160883110359040,543803256,Fernando Pérez Hernández🇨🇺,FernandoPerezHd,Cienfuegos,No soy Político ni pretendo convertirme en un ...,NaN,3971,3540,2012-04-02,...,absent,absent,0,0,0,0,"['Cuba', 'CubaVa', 'CubaSi']",['http://ln.is/bit.ly/Z3ATI'],[587848359],RT : Cuba CubaVa Participa en el tuitazo mundi...
4,642077163173347329,543803256,Fernando Pérez Hernández🇨🇺,FernandoPerezHd,Cienfuegos,No soy Político ni pretendo convertirme en un ...,NaN,3971,3540,2012-04-02,...,absent,absent,0,0,0,0,"['cuba', 'YoSoyCuba', 'PensandoAmericas']",['http://bit.ly/1Og4ZX3'],[602879083],RT : cuba YoSoyCuba Expondrá Esterio Sergura e...


In [122]:
# Remove tweets with length < 5
filtered_control_df['tweet_length_cleaned'] = filtered_control_df['full_text_cleaned'].apply(lambda x: len(x))
filtered_treated_df['tweet_length_cleaned'] = filtered_treated_df['tweet_text_cleaned'].apply(lambda x: len(x))

filtered_control_df_after_cleaning = filtered_control_df[filtered_control_df['tweet_length_cleaned'] > 5]
filtered_treated_df_after_cleaning = filtered_treated_df[filtered_treated_df['tweet_length_cleaned'] > 5]


/tmp/ipykernel_776725/2726823249.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_control_df['tweet_length_cleaned'] = filtered_control_df['full_text_cleaned'].apply(lambda x: len(x))
/tmp/ipykernel_776725/2726823249.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_treated_df['tweet_length_cleaned'] = filtered_treated_df['tweet_text_cleaned'].apply(lambda x: len(x))


In [123]:
control_users_tweets = filtered_control_df_after_cleaning.userid.unique()
treated_users_tweets = filtered_treated_df_after_cleaning.userid.unique()

assert len(np.union1d(control_users_tweets, treated_users_tweets)) == len(valid_user_ids), len(valid_user_ids)-len(np.union1d(control_users_tweets, treated_users_tweets))


In [129]:
filtered_control_df_after_cleaning.groupby('userid').count()['id'].describe()


count   4377.000
mean      93.077
std      154.417
min       18.000
25%       47.000
50%       85.000
75%       99.000
max     6396.000
Name: id, dtype: float64

In [130]:
filtered_treated_df_after_cleaning.groupby('userid').count()['tweetid'].describe()


count      429.000
mean     11082.014
std      25913.050
min         28.000
25%        812.000
50%       3785.000
75%      10577.000
max     268616.000
Name: tweetid, dtype: float64

In [126]:
filtered_control_df_after_cleaning['tweet_length_cleaned'].describe()


count   407396.000
mean        93.929
std         35.519
min          6.000
25%         70.000
50%        102.000
75%        124.000
max        482.000
Name: tweet_length_cleaned, dtype: float64

In [127]:
filtered_treated_df_after_cleaning['tweet_length_cleaned'].describe()

count   4754184.000
mean        100.104
std          41.242
min           6.000
25%          81.000
50%         110.000
75%         123.000
max         380.000
Name: tweet_length_cleaned, dtype: float64

#### Analysis by language

In [162]:
abs_occurrence_lang = filtered_treated_df_after_cleaning['tweet_language'].value_counts()
rel_perc_lang = abs_occurrence_lang / filtered_treated_df_after_cleaning['tweet_language'].value_counts().sum()
rel_perc_lang = rel_perc_lang.rename('perc')
pd.concat([abs_occurrence_lang, rel_perc_lang], axis=1)


,count,perc
tweet_language,,
es,4353175,0.935
en,157291,0.034
und,89701,0.019
pt,26498,0.006
it,6771,0.001
fr,4046,0.001
de,2883,0.001
ca,2707,0.001
ru,1596,0.000


In [163]:
abs_occurrence_lang = filtered_control_df_after_cleaning['lang'].value_counts()
rel_perc_lang = abs_occurrence_lang / filtered_control_df_after_cleaning['lang'].value_counts().sum()
rel_perc_lang = rel_perc_lang.rename('perc')
pd.concat([abs_occurrence_lang, rel_perc_lang], axis=1)


,count,perc
lang,,
es,330795,0.812
en,47215,0.116
und,11685,0.029
pt,5477,0.013
th,1808,0.004
ko,1764,0.004
ca,1353,0.003
fr,1273,0.003
in,1166,0.003


# Using XLM-Roberta to embed each tweet

In [168]:
from tqdm.notebook import tqdm

In [175]:
treated_users_embeddings = []
tweet_text_colname = "full_text_cleaned"  # "tweet_text_cleaned"
for userid in control_users_tweets:
    tweets_text = filtered_control_df_after_cleaning[filtered_control_df_after_cleaning['userid'] == userid][tweet_text_colname]
    break


In [197]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import torch
from transformers import AutoTokenizer, XLMRobertaModel

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large")
model = XLMRobertaModel.from_pretrained("xlm-roberta-large")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 1024, padding_idx=1)
    (position_embeddings): Embedding(514, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-23): 24 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwi

In [199]:
# Function to get [CLS] token embedding for a given text
def get_cls_embedding(text, tokenizer, model, device):
    # Tokenize input text
    sentence = torch.tensor([tokenizer.encode(text)]).to(device)
    # Get model output
    with torch.no_grad():
        output = model(sentence)
    # Extract last layer token embeddings
    hidden_states = output[-1]
    # Extract [CLS] token embeddings
    return hidden_states[0]

sample_text = filtered_control_df_after_cleaning['full_text_cleaned'].iloc[0]
output = get_cls_embedding(sample_text, tokenizer, model, device)


In [ ]:
user_tweets_embeddings = []
for tweet_text


In [201]:
output.shape

torch.Size([1024])

In [195]:
embedding_output.shape

torch.Size([1024])

In [191]:
output

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.5563, -0.8087]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)